In [2]:
import csv
import random

In [3]:
# define the training data
# training_data = [
#     ['Green', 3, 'Apple'],
#     ['Yellow', 3, 'Apple'],
#     ['Red', 1, 'Grape'],
#     ['Red', 1, 'Grape'],
#     ['Yellow', 3, 'Lemon'],
# ]

In [4]:
csv_data = []

with open('../datasets/iris.csv') as csv_file:
    readCSV = csv.reader(csv_file, delimiter=",")
    for row in readCSV:
        csv_data.append(row)

In [5]:
header = csv_data[0]

training_data = []
for i in range(1, len(csv_data)):
    train_row = [float(csv_data[i][j]) for j in range(len(csv_data[0]) - 1)]
    train_row.append(csv_data[i][-1])
    
    training_data.append(train_row)

In [6]:
# splitting the training and testing data
idx = list(range(len(training_data)))
    
random.seed(7382)
random.shuffle(idx)
    
train_size = int(0.77 * len(training_data))
test_size = len(training_data) - train_size

train_idx = idx[:train_size] 
test_idx = idx[train_size:]
    
train_data = [training_data[i] for i in train_idx]
test_data = [training_data[i] for i in test_idx]

print(train_data[0])
print(len(train_data))
print(test_data[0])
print(len(test_data))

[5.1, 3.5, 1.4, 0.2, 'Iris-setosa']
115
[5.8, 2.7, 5.1, 1.9, 'Iris-virginica']
35


In [7]:
print(len(train_data))
print(len(train_data[0]))
print(train_data[:5][:])

115
5
[[5.1, 3.5, 1.4, 0.2, 'Iris-setosa'], [6.2, 2.8, 4.8, 1.8, 'Iris-virginica'], [6.3, 2.9, 5.6, 1.8, 'Iris-virginica'], [5.6, 3.0, 4.1, 1.3, 'Iris-versicolor'], [4.9, 2.4, 3.3, 1.0, 'Iris-versicolor']]


In [8]:
# the column names for the data
# header = ["color", "diameter", "label"]

In [9]:
header

['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth', 'Name']

In [10]:
def unique_vals(rows, col):
    """
    Find unique values in a column
    """
    return set([row[col] for row in rows])

In [11]:
unique_vals(train_data, 1)

{2.2,
 2.3,
 2.4,
 2.5,
 2.6,
 2.7,
 2.8,
 2.9,
 3.0,
 3.1,
 3.2,
 3.3,
 3.4,
 3.5,
 3.6,
 3.7,
 3.8,
 3.9,
 4.0,
 4.1,
 4.2,
 4.4}

In [12]:
def class_counts(rows):
    """
    Counts the number of each type of class in the dataset
    """
    counts = {}
    for row in rows:
        label = row[-1]
        if label not in counts:
            counts[label] = 0
        counts[label] += 1
    return counts

In [13]:
class_counts(train_data)

{'Iris-setosa': 39, 'Iris-virginica': 35, 'Iris-versicolor': 41}

In [14]:
def is_numeric(value):
    """Tests if a value is numerics"""
    return isinstance(value, int) or isinstance(value, float)

In [15]:
is_numeric(89.98)

True

In [16]:
class Question:
    """A question asked to partition the dataset
    
    Stores a 'column number' along with the 'column value'
    """

    def __init__(self, column, value):
        self.column = column
        self.value = value
        
    def match(self, example):
        # compare the feature value in question to the 
        # feature value in example
        val = example[self.column]
        if is_numeric(val):
            return val >= self.value
        else:
            return val == self.value
        
    def __repr__(self):
        # print the question in readable format
        condition = "=="
        if is_numeric(self.value):
            condition = ">="
        return "Is %s %s %s?" %(header[self.column], 
                               condition, str(self.value))

In [17]:
Question(1, 3)

Is SepalWidth >= 3?

In [18]:
# Question(0, 'Green')

In [19]:
def partitions(rows, question):
    """Partition the dataset into 'true rows' or 
    'false rows' based on the matching with question
    """
    
    true_rows, false_rows = [], []
    for row in rows:
        if question.match(row):
            true_rows.append(row)
        else:
            false_rows.append(row)
    return true_rows, false_rows

In [20]:
question = Question(1, 3)
question

Is SepalWidth >= 3?

In [21]:
true_rows, false_rows = partitions(train_data, question)

In [22]:
print(true_rows[:5])
print(len(true_rows))

[[5.1, 3.5, 1.4, 0.2, 'Iris-setosa'], [5.6, 3.0, 4.1, 1.3, 'Iris-versicolor'], [6.0, 3.0, 4.8, 1.8, 'Iris-virginica'], [4.8, 3.1, 1.6, 0.2, 'Iris-setosa'], [6.1, 3.0, 4.6, 1.4, 'Iris-versicolor']]
69


In [23]:
print(false_rows[:5])
print(len(false_rows))

[[6.2, 2.8, 4.8, 1.8, 'Iris-virginica'], [6.3, 2.9, 5.6, 1.8, 'Iris-virginica'], [4.9, 2.4, 3.3, 1.0, 'Iris-versicolor'], [6.0, 2.2, 4.0, 1.0, 'Iris-versicolor'], [4.5, 2.3, 1.3, 0.3, 'Iris-setosa']]
46


In [24]:
def gini(rows):
    """Calculates the Gini Impurity for a list of rows"""
    
    counts = class_counts(rows)
    impurity = 1
    for lbl in counts:
        prob_lbl = counts[lbl] / float(len(rows))
        impurity -= prob_lbl**2
    return impurity

In [25]:
mixed_dataset = [['Iris-versicolor'],
                  ['Iris-versicolor'],
                  ['Iris-setosa'],
                  ['Iris-versicolor'],
                  ['Iris-versicolor']]
gini(mixed_dataset)

0.31999999999999984

In [26]:
def info_gain(left, right, current_uncertainty):
    """
    Calculate information gain i.e.
    uncertainty of starting node minus the weighted impurity
    of two child nodes
    """
    p = float(len(left)) / (len(left) + len(right))
    return current_uncertainty - p*gini(left) - (1-p)*gini(right)

In [27]:
current_uncertainty = gini(train_data)
current_uncertainty

0.6652551984877126

In [28]:
true_rows, false_rows = partitions(train_data, Question(0, 5))
info_gain(true_rows, false_rows, current_uncertainty)

0.08680767982280255

In [29]:
print(len(true_rows))
print(true_rows[:5])

99
[[5.1, 3.5, 1.4, 0.2, 'Iris-setosa'], [6.2, 2.8, 4.8, 1.8, 'Iris-virginica'], [6.3, 2.9, 5.6, 1.8, 'Iris-virginica'], [5.6, 3.0, 4.1, 1.3, 'Iris-versicolor'], [6.0, 2.2, 4.0, 1.0, 'Iris-versicolor']]


In [30]:
print(len(false_rows))
print(false_rows[:5])

16
[[4.9, 2.4, 3.3, 1.0, 'Iris-versicolor'], [4.5, 2.3, 1.3, 0.3, 'Iris-setosa'], [4.8, 3.1, 1.6, 0.2, 'Iris-setosa'], [4.3, 3.0, 1.1, 0.1, 'Iris-setosa'], [4.6, 3.6, 1.0, 0.2, 'Iris-setosa']]


In [31]:
def find_best_split(rows):
    """
    Find the best splitting criteria - best question
    and the information gain
    """
    
    best_gain = 0
    best_question = None
    current_uncertainty = gini(rows)
    n_features = len(rows[0]) - 1
    
    for col in range(n_features):
        
        values = set([row[col] for row in rows])
        
        for val in values:
            
            question = Question(col, val)
            true_rows, false_rows = partitions(rows, question)        
            
            if len(true_rows) == 0 or len(false_rows) == 0:
                continue # no need to split
                
            gain = info_gain(true_rows, false_rows, current_uncertainty)
            
            if gain >= best_gain:
                best_gain, best_question = gain, question
    
    return best_gain, best_question

In [32]:
best_gain, best_question = find_best_split(train_data)
best_question

Is PetalWidth >= 1.0?

In [33]:
class Leaf:
    """
    A leaf node classifies the data
    Stores the number of times a 'class' appears in the 
    rows from the training data that reach this leaf
    """
    
    def __init__(self, rows):
        self.predictions = class_counts(rows)

In [34]:
class DecisionNode:
    """
    A decision node asks a question
    Holds a reference to the question
    and to the two child nodes
    """
    
    def __init__(self, question, true_branch, false_branch):
        self.question = question
        self.true_branch = true_branch
        self.false_branch = false_branch    

In [35]:
def build_tree(rows, depth=0):
    """
    Builds the tree
    """
    
    # get the gain and the question with highest gain
    gain, question = find_best_split(rows)
    
    # all classes are the same
    if gain == 0 or depth == 3:
        return Leaf(rows)
    
    true_rows, false_rows = partitions(rows, question)
    
    # build the true subtree
    true_branch = build_tree(true_rows, depth=depth+1)
    
    # build the false subtree
    false_branch = build_tree(false_rows, depth=depth+1)
    
    # the Question node i.e question to ask at this node
    return DecisionNode(question, true_branch, false_branch)

In [36]:
def print_tree(node, spacing=""):
    """
    Print the tree
    """
    
    if isinstance(node, Leaf):
        print(spacing + "Predict", node.predictions)
        return

    print(spacing + str(node.question))
    
    print(spacing + '--> True:')
    print_tree(node.true_branch, spacing + "  ")
    
    print(spacing + '--> False:')
    print_tree(node.false_branch, spacing + "  ")

In [37]:
my_tree = build_tree(train_data)

In [45]:
print_tree(my_tree)

Is PetalWidth >= 1.0?
--> True:
  Is PetalWidth >= 1.8?
  --> True:
    Is PetalLength >= 4.9?
    --> True:
      Predict {'Iris-virginica': 31}
    --> False:
      Predict {'Iris-virginica': 2, 'Iris-versicolor': 1}
  --> False:
    Is PetalLength >= 5.0?
    --> True:
      Predict {'Iris-virginica': 2, 'Iris-versicolor': 1}
    --> False:
      Predict {'Iris-versicolor': 39}
--> False:
  Predict {'Iris-setosa': 39}


In [39]:
def classify(row, node):
    """
    classify a given test data - row
    the root of decision tree - node
    """
    
    if isinstance(node, Leaf):
        return node.predictions
    
    if node.question.match(row):
        return classify(row, node.true_branch)
    else:
        return classify(row, node.false_branch)

In [40]:
classify(test_data[3], my_tree)

{'Iris-versicolor': 39}

In [41]:
def print_leaf(counts):
    total = sum(counts.values()) * 1.0
    probs = {}
    for lbl in counts.keys():
        probs[lbl] = str(int(counts[lbl] / total * 100)) + "%"
    return probs

In [42]:
print_leaf(classify(test_data[1], my_tree))

{'Iris-versicolor': '100%'}

In [43]:
# testing_data = [
#     ['Green', 3, 'Apple'],
#     ['Yellow', 4, 'Apple'],
#     ['Red', 2, 'Grape'],
#     ['Red', 1, 'Grape'],
#     ['Yellow', 3, 'Lemon'],
# ]

In [44]:
for row in test_data:
    print("Actual: %s. Predicted: %s" %(row[-1], print_leaf(classify(row, my_tree))))

Actual: Iris-virginica. Predicted: {'Iris-virginica': '100%'}
Actual: Iris-versicolor. Predicted: {'Iris-versicolor': '100%'}
Actual: Iris-virginica. Predicted: {'Iris-virginica': '66%', 'Iris-versicolor': '33%'}
Actual: Iris-versicolor. Predicted: {'Iris-versicolor': '100%'}
Actual: Iris-virginica. Predicted: {'Iris-versicolor': '100%'}
Actual: Iris-setosa. Predicted: {'Iris-setosa': '100%'}
Actual: Iris-virginica. Predicted: {'Iris-virginica': '100%'}
Actual: Iris-setosa. Predicted: {'Iris-setosa': '100%'}
Actual: Iris-versicolor. Predicted: {'Iris-versicolor': '100%'}
Actual: Iris-versicolor. Predicted: {'Iris-versicolor': '100%'}
Actual: Iris-virginica. Predicted: {'Iris-virginica': '66%', 'Iris-versicolor': '33%'}
Actual: Iris-virginica. Predicted: {'Iris-virginica': '100%'}
Actual: Iris-versicolor. Predicted: {'Iris-versicolor': '100%'}
Actual: Iris-setosa. Predicted: {'Iris-setosa': '100%'}
Actual: Iris-versicolor. Predicted: {'Iris-versicolor': '100%'}
Actual: Iris-setosa. Pred